In [27]:
list_of_sentences = []

with open("merged_pretrain_data_cleaned.txt", "r", encoding="utf-8") as f:
    for line in f:
        cleaned_line = line.strip()
        list_of_sentences.append(cleaned_line)

In [28]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer

llama32tokenizer = Tokenizer(BPE(unk_token="<unk>"))
trainer = BpeTrainer(special_tokens=["<unk>", "<pad>", "<bos>", "<eos>"], vocab_size=35000)

llama32tokenizer.train_from_iterator(list_of_sentences, trainer)

In [29]:
from transformers import PreTrainedTokenizerFast

fast_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=llama32tokenizer,
    pad_token="<pad>",
    bos_token="<bos>",
    eos_token="<eos>"
)

In [32]:
fast_tokenizer.save_pretrained("./my_tokenizer")


('./my_tokenizer/tokenizer_config.json',
 './my_tokenizer/special_tokens_map.json',
 './my_tokenizer/tokenizer.json')

In [ ]:
from transformers import PreTrainedTokenizerFast

fast_tokenizer = PreTrainedTokenizerFast.from_pretrained("my_tokenizer")

In [113]:
fast_tokenizer.vocab_size

35000

In [19]:
list_of_sentences = []

with open("merged_pretrain_data_cleaned.txt", "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f]

In [20]:
lines[:10]

['muhteşem bir filmdi kesinlikle izlenmeye değer olarak görüyorum görüntü kalitesi ve kurgu çok iyiydi verilen puanlardan daha fazlasını hakettiğini düşünüyorum',
 'İlk defa küfürsüz bir komedi filmi bence diğerleri kıskanıyorlar ben cem yılmazın filminde bile bu kadar eğlenmedim bu da aynen ıssız adam filmine benzeyecek giden herkes tavsiye ediyor önemli olan seyircinin beğenmnesi bence en önemli juri seyirci çıkan herkes filme 10 numara diyordu hele gürgen öz süperrrrrrrrrrrrrrr',
 'müthiş',
 'gerçekten çok daha iyisini bekliyodum evet erotik olması için çıplaklık olmasına gerek yok ama konu da beni vurmadı açıkçası yine de her Kubrick filmi gibi izlemeye değer çok şey bekleyip sadece beğendiğim bi film',
 'unal15 arkadaşıma katılıyorum gereksiz bi film SAW serisi sevenleri kızdıracak bi film fragmanından berbat bi şey olduğu belli 0 1 10 bizim burda bir gün erkenden gösterime giriği için izleyipte puan veriyorum izlemediğim filme puan vermem adetim değildir',
 'freestyle79 kardeşim 

In [29]:
fast_tokenizer.encode(lines[0])

[3985,
 5703,
 2957,
 14821,
 2192,
 18067,
 4873,
 4247,
 4419,
 4801,
 11829,
 4460,
 9026,
 2698,
 23459,
 2475,
 6275,
 4952]

In [33]:
len(lines)

1742414

In [ ]:
from collections import defaultdict

dicti = defaultdict(int)

line_number = 0
for line in lines:
    tokens = fast_tokenizer.encode(line)
    for token in tokens:
        dicti[token] += 1
    line_number += 1
    print(f"Processed line number: {line_number}")

In [ ]:
dicti

In [43]:
#sort dicti by values in descending order
dicti = dict(sorted(dicti.items(), key=lambda item: item[1], reverse=True))

In [44]:
import json

with open("token_frequency.json","w") as f :
    json.dump(dicti, f)

In [51]:
dicti[1279]

2

In [ ]:
ids        token_num 
617-1094 = 478 
1279-1751 =473 
1800-1829 =30 
1846-1892 =47 

total = 1028 token
2232 token by freq


In [52]:
delete_ids= list(range(617,1095)) + list(range(1279,1752)) + list(range(1800,1830)) + list(range(1846,1893))

In [54]:
import json

with open("token_frequency.json","w") as f :
    json.dump(dicti, f)


In [70]:
lst=[]

k=0
for i in reversed(list(dicti.keys())):
    if (i not in delete_ids) and i>=24:
        lst.append(i)
        k+=1
    if k==1204:
        break

In [71]:
len(lst)

1204

In [73]:
#concat delete_ids and lst
final_ids = delete_ids + lst

In [74]:
len(final_ids)

2232

delete tokens from vocab and merges

In [5]:
import json
with open("./my_tokenizer/tokenizer.json","r") as f:
    tokenizer_json = json.load(f)

In [90]:
new_vocab = {}
new_id = 0

for token, old_id in tokenizer_json["model"]["vocab"].items():
    if old_id in final_ids:  # skip tokens we want to delete
        continue
    new_vocab[token] = new_id  # assign new consecutive ID
    new_id += 1

In [117]:
len(list(new_vocab))

32768

In [136]:
len(tokenizer_json["model"]["merges"])

32313

In [118]:
new_merges=[]

for couple_list in tokenizer_json["model"]["merges"]:
    if couple_list[0] in new_vocab and couple_list[1] in new_vocab:
        new_merges.append(couple_list)
        

In [120]:
len(new_merges)

32313

In [144]:
tokenizer_json["model"]["vocab"] =new_vocab
tokenizer_json["model"]["merges"] = new_merges

In [122]:
with open("./my_tokenizer/tokenizer.json", "w", encoding="utf-8") as f:
    json.dump(tokenizer_json, f, ensure_ascii=False, indent=2)

In [1]:
from transformers import PreTrainedTokenizerFast

fast_tokenizer = PreTrainedTokenizerFast.from_pretrained("my_tokenizer")

Exception: Token `avsiye ` out of vocabulary at line 162081 column 1

In [131]:
list(tokenizer_json["model"]["vocab"].keys()).count("`avsiye `")

0

In [ ]:
for listt in tokenizer_json["model"]["merges"]:
    if listt[0]=="`avsiye" or listt[1]==" `":
        print("found")

In [ ]:
fast_tokenizer.push_to_hub("AhmetSemih/merged_dataset-32k-bpe-tokenizer")